<a href="https://colab.research.google.com/github/fspanhoff/sgf_converter/blob/master/sgf_converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load a sgf file played against Leela @ online-go and make it accessible in a sgf editor with variation trees



In [0]:
import re
from google.colab import files
from string import ascii_lowercase

**Go Move annotation:**

Go is played on a 19x19 board. Moves are traditionally written down


**On the sgf coordinate system:**
The first letter designates the column (left to right), the second the row (top to bottom).

The author intentionally broke with the tradition of labeling moves (and points) with letters "A"-"T" (excluding "i") and numbers 1-19. 
Two lowercase letters in the range "a"-"s" were used instead, for reasons of simplicity and compactness.

In [0]:
def cord_to_sgf(x_axis, y_axis):
  '''transforms a move written in traditional notation to sgf format
  
  Input is two characters x_axis 'A'-'T' and y-axis: '1'-'19'
  
  Output is a sgf format between brackets:
  
  e.g. 'D','4' will return: '[dp]'
  '''
  sgf_format = ascii_lowercase[0:19]
  traditional_format = ascii_lowercase[0:20].replace('i', '').upper() 
  x = sgf_format[traditional_format.index(x_axis)]
  y = sgf_format[19-int(y_axis)]  # traditonal notation counts from right to left for the y_axis 
  return ('['+x+y+']')

In [0]:
def comment_to_sgf(line):
  '''transform comments of Leela on what it would have played into a sgf variation
  
  Input is a string from the sgf file with leela's comments on what it thinks was the best move
  Output is a string in sgf format to be added as a variation
  
  e.g.  input:
        'C[15bTurboLeela: Winrate: 2.38%, Visits: 5716, Playouts: 5715. From move 5: R3 R4 Q3 O3 P3 P4 O2 C17 C16 D17 F17 E17 E16'
  returns: 
        '(;B[qp];W[pq];B[nq];W[oq];B[op];W[nr];B[cc];W[cd];B[dc];W[fc];B[ec];W[ed])\n'
  '''
  moves = re.findall(r'([A-Z])(\d{1,2})',line)
  del moves[0] # Leela repeats the last move, which is already played
  variation = ''
  color = [';B',';W']
  for idx, move in enumerate(moves):
    variation +=  color[idx % 2 ] + cord_to_sgf(move[0],move[1])
  return '(' + variation + ')' + '\n'

In [0]:
uploaded = files.upload()

Saving 16815395-090-15bTurboLeela-FrederikSpanhoff.sgf to 16815395-090-15bTurboLeela-FrederikSpanhoff.sgf


In [0]:
filename = list(uploaded.keys())[0]
sgf = "".join(map(chr, uploaded[filename]))            # convert bytes to string
index_firstmove = re.search('(;B\[|;W\[)',sgf).start() 
sgf_per_line = sgf[index_firstmove:].splitlines()      # all the go moves without the header part

In [0]:
main_branch  = ''; variations = ''

for linenr, line in enumerate(sgf_per_line):
    if line.startswith(';B'):
      main_branch += line + '\n'
    if line.startswith(';W'):
      main_branch += line + '\n' 
      if sgf_per_line[linenr+3].startswith(';B'): # if it is the last move we don't add a variation
        main_branch += '('
    if line.startswith('C') and sgf_per_line[linenr+2].startswith(';B'):
      variations = comment_to_sgf(line) + ')' + variations 

full_sgf = sgf[:index_firstmove] + main_branch + ')' + variations

In [0]:
with open(filename, 'w') as f:
  f.write(full_sgf)

files.download(filename)